In [1]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import numpy as np

In [30]:
# Establish a connection to the SQLite database
conn = sqlite3.connect('Data/users_data.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Query to retrieve table names from the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the table names
table_names = cursor.fetchall()

# Print the table names
for name in table_names:
    print(name[0])

Users


In [31]:
# Read data from the database into a DataFrame
users = pd.read_sql('SELECT * FROM Users', conn)

# Close the cursor and connection
cursor.close()
conn.close()

In [32]:
users.head(3)

,id,username,age,gender,fav_entertainment,least_fav_entertainment,likes,dislikes,movie_watching_freq,show_watching_freq,reading_freq
0,1,Krystal9132,22,F,"Movies, Tv-shows",Books,History,"Thriller, Documentary",Daily,Daily,Never
1,2,Rita4030,31,F,"Tv-shows, Movies",None,"Classics, Documentary",Comedy,Daily,Rarely,Rarely
2,3,Jacqueline534,68,M,"Movies, Books",Tv-shows,"Animation, Adventure, Sci-Fi","Documentary, Drama, Classics, Thriller, Action",Daily,Rarely,Monthly


In [33]:
users_encoded = users

In [34]:
users_encoded = users_encoded.drop(columns=['id', 'username'])

In [35]:
users_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   age                      10000 non-null  int64 
 1   gender                   10000 non-null  object
 2   fav_entertainment        10000 non-null  object
 3   least_fav_entertainment  10000 non-null  object
 4   likes                    10000 non-null  object
 5   dislikes                 10000 non-null  object
 6   movie_watching_freq      10000 non-null  object
 7   show_watching_freq       10000 non-null  object
 8   reading_freq             10000 non-null  object
dtypes: int64(1), object(8)
memory usage: 703.3+ KB


In [36]:
def convert_to_binary(value):
    if value == 'F':
        return 0
    elif value == 'M':
        return 1
    else:
        return np.nan

In [37]:
users_encoded['gender'] = users_encoded['gender'].apply(lambda x: convert_to_binary(x))


In [38]:
users_encoded

,age,gender,fav_entertainment,least_fav_entertainment,likes,dislikes,movie_watching_freq,show_watching_freq,reading_freq
0,22,0,"Movies, Tv-shows",Books,History,"Thriller, Documentary",Daily,Daily,Never
1,31,0,"Tv-shows, Movies",None,"Classics, Documentary",Comedy,Daily,Rarely,Rarely
2,68,1,"Movies, Books",Tv-shows,"Animation, Adventure, Sci-Fi","Documentary, Drama, Classics, Thriller, Action",Daily,Rarely,Monthly
3,46,1,Books,Tv-shows,"Classics, History, Action, Sci-Fi, Fantasy","Documentary, Action, History",Rarely,Monthly,Monthly
4,36,1,"Books, Movies",Tv-shows,"Classics, Comedy",Comedy,Weekly,Never,Daily
...,...,...,...,...,...,...,...,...,...
9995,64,0,Tv-shows,None,"Sci-Fi, Horror, Mystery, Documentary, Animation","Fiction, Mystery, Classics, Comedy, Horror",Monthly,Weekly,Daily
9996,59,1,Tv-shows,Movies,"Comedy, Mystery, Fantasy, Drama","Fantasy, Romance, Sci-Fi, Animation, Action",Monthly,Never,Rarely
9997,71,0,"Books, Movies",None,"Drama, Horror, Classics","Fantasy, Romance, Comedy",Weekly,Daily,Daily
9998,38,0,Movies,Books,"Thriller, Mystery, Action, Documentary","Mystery, Action, History",Rarely,Monthly,Daily


In [39]:
# Preprocess the columns to standardize the combinations
users_encoded['fav_entertainment'] = users_encoded['fav_entertainment'].apply(lambda x: ', '.join(sorted(x.split(', '))))
users_encoded['least_fav_entertainment'] = users_encoded['least_fav_entertainment'].apply(lambda x: ', '.join(sorted(x.split(', '))))

# One-hot encode 'fav_entertainment' column
fav_encoded = users_encoded['fav_entertainment'].str.get_dummies(', ').add_prefix('fav_')

# One-hot encode 'least_fav_entertainment' column
least_fav_encoded = users_encoded['least_fav_entertainment'].str.get_dummies(', ').add_prefix('least_fav_')

# Concatenate the new one-hot encoded columns with the original DataFrame
users_encoded = pd.concat([users_encoded, fav_encoded, least_fav_encoded], axis=1)

# Drop the original columns if needed
users_encoded.drop(['fav_entertainment', 'least_fav_entertainment'], axis=1, inplace=True)

In [40]:
# Preprocess the columns to standardize the combinations
users_encoded['likes'] = users_encoded['likes'].apply(lambda x: ', '.join(sorted(x.split(', '))))
users_encoded['dislikes'] = users_encoded['dislikes'].apply(lambda x: ', '.join(sorted(x.split(', '))))

# One-hot encode 'likes' column
likes_encoded = users_encoded['likes'].str.get_dummies(', ').add_prefix('likes_')

# One-hot encode 'dislikes' column
dislikes_encoded = users_encoded['dislikes'].str.get_dummies(', ').add_prefix('dislikes_')

# Concatenate the new one-hot encoded columns with the original DataFrame
users_encoded = pd.concat([users_encoded, likes_encoded, dislikes_encoded], axis=1)

# Drop the original columns if needed
users_encoded.drop(['likes', 'dislikes'], axis=1, inplace=True)

In [45]:
# Define the encoding map
encoding_map = {'Never': 0, 'Rarely': 1, 'Monthly': 2, 'Weekly': 3, 'Daily': 4}

# Apply the mapping to encode the variables in users_encoded
users_encoded['movie_watching_freq'] = users_encoded['movie_watching_freq'].map(encoding_map)
users_encoded['show_watching_freq'] = users_encoded['show_watching_freq'].map(encoding_map)
users_encoded['reading_freq'] = users_encoded['reading_freq'].map(encoding_map)

In [46]:
users_encoded

,age,gender,movie_watching_freq,show_watching_freq,reading_freq,fav_Books,fav_Movies,fav_Tv-shows,least_fav_Books,least_fav_Movies,...,dislikes_Documentary,dislikes_Drama,dislikes_Fantasy,dislikes_Fiction,dislikes_History,dislikes_Horror,dislikes_Mystery,dislikes_Romance,dislikes_Sci-Fi,dislikes_Thriller
0,22,0,4,4,0,0,1,1,1,0,...,1,0,0,0,0,0,0,0,0,1
1,31,0,4,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,68,1,4,1,2,1,1,0,0,0,...,1,1,0,0,0,0,0,0,0,1
3,46,1,1,2,2,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
4,36,1,3,0,4,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,64,0,2,3,4,0,0,1,0,0,...,0,0,0,1,0,1,1,0,0,0
9996,59,1,2,0,1,0,0,1,0,1,...,0,0,1,0,0,0,0,1,1,0
9997,71,0,3,4,4,1,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
9998,38,0,1,2,4,0,1,0,1,0,...,0,0,0,0,1,0,1,0,0,0


In [48]:
users_encoded.columns

Index(['age', 'gender', 'movie_watching_freq', 'show_watching_freq',
       'reading_freq', 'fav_Books', 'fav_Movies', 'fav_Tv-shows',
       'least_fav_Books', 'least_fav_Movies', 'least_fav_None',
       'least_fav_Tv-shows', 'likes_Action', 'likes_Adventure',
       'likes_Animation', 'likes_Classics', 'likes_Comedy',
       'likes_Documentary', 'likes_Drama', 'likes_Fantasy', 'likes_Fiction',
       'likes_History', 'likes_Horror', 'likes_Mystery', 'likes_Romance',
       'likes_Sci-Fi', 'likes_Thriller', 'dislikes_Action',
       'dislikes_Adventure', 'dislikes_Animation', 'dislikes_Classics',
       'dislikes_Comedy', 'dislikes_Documentary', 'dislikes_Drama',
       'dislikes_Fantasy', 'dislikes_Fiction', 'dislikes_History',
       'dislikes_Horror', 'dislikes_Mystery', 'dislikes_Romance',
       'dislikes_Sci-Fi', 'dislikes_Thriller'],
      dtype='object')

In [ ]:
dispersion = []
for k in range(1, 20):
    kmeans = KMeans(n_clusters=k, random_state=42).fit(data)
    dispersion.append(kmeans.inertia_)

In [ ]:
plt.plot(range(1, 20), dispersion, marker='o', color='darkorange')
plt.xlabel('Number of clusters')
plt.ylabel('Dispersion (inertia)')
plt.show()

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
silhouette_scores = compute_silhouette_scores(data.values, range_n_clusters)
print(silhouette_scores)

In [ ]:
plt.plot(range(2, 15), silhouette_scores, marker='o', color='darkorange')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Scores')
plt.show()